#Import and Initial Mount Disk


In [1]:
# install library
!pip install -U tensorflow-addons
!pip install facenet-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.8 MB/s 


In [ ]:
import os
# Mount drive
from google.colab import drive
drive.mount("/content/drive")
path = "/content/drive/My Drive/Colab Notebooks/FaceMaskRecognize"
os.chdir(path)

import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras import models, layers, metrics, optimizers, Model
from functools import partial
import matplotlib.pyplot as plt
import numpy as np
import cv2
import math
import io
import pickle
import tensorflow_datasets as tfds
import random
from train.Net import InceptionResNetV1
from train.FaceNet import FaceNetModel,call_instance_FaceNet_with_custom, call_instance_FaceNet_without_custom,call_instance_FaceNet_with_last_isDense, convert_train_model_to_embedding
from train.Classify import Classify
from tool.FormatFunction import FormatFunction
from tool.FileFunction import FileFunction
from tool.GlobalValue import GlobalValue

Mounted at /content/drive


# Train

## Init value

In [ ]:
READ_RAW_DATA_THEN_SAVE = False
global_value = GlobalValue(image_size=[110,110], batch_size = 512, shuffle_size = 1000, ratio_train = 0.8, ratio_test = 0.1, ratio_valid = 0.1, epochs = 40, small_epochs = 50,
                           image_each_class = 15)
format_function = FormatFunction(global_value)

if READ_RAW_DATA_THEN_SAVE: 
  label_dict = format_function.get_label_dict(os.path.join(os.getcwd(),"align_image"))
  path = os.path.join(os.getcwd(),"src","data","label_dict.pkl")
  with open(path, 'wb') as file:
    pickle.dump(label_dict, file)
path = os.path.join(os.getcwd(),"src","data","label_dict.pkl")
with open(path, 'rb') as f:
  label_dict = pickle.load(f)

file_function = FileFunction()


## Load data 

In [ ]:

#Save data path to file to read faster
if READ_RAW_DATA_THEN_SAVE:
  path_image_align  = file_function.get_data_path_by_dictionary(os.path.join(os.getcwd(),"align_image"))
  path = os.path.join(os.getcwd(),"src","data","align_data_path.pkl")
  with open(path, 'wb') as file:
      pickle.dump(path_image_align, file)

  path_image_mask  = file_function.get_data_path_by_dictionary(os.path.join(os.getcwd(),"face+mask_image"))
  path = os.path.join(os.getcwd(),"src","data","face+mask_data_path.pkl")
  with open(path, 'wb') as file:
      pickle.dump(path_image_mask, file)

# Read data path from file
path = os.path.join(os.getcwd(),"src","data","align_data_path.pkl")
with open(path, 'rb') as f:
    path_image_align = pickle.load(f)
    path_image_align = file_function.get_data_path_with_limit(path_image_align,global_value.IMAGE_EACH_CLASS)
path = os.path.join(os.getcwd(),"src","data","face+mask_data_path.pkl")
with open(path, 'rb') as f:
    path_image_mask = pickle.load(f)
    path_image_mask = file_function.get_data_path_with_limit(path_image_mask,global_value.IMAGE_EACH_CLASS)
# Combine data path

path_image_align.extend(path_image_mask)
random.shuffle(path_image_align)
label_index =list()
for path in path_image_align:
  label = path.split("/")[-2]
  label = label_dict[label]
  label_index.append(label)
path_dataset = tf.data.Dataset.from_tensor_slices(path_image_align)
label_dataset = tf.data.Dataset.from_tensor_slices(label_index)
origin_dataset = tf.data.Dataset.zip((path_dataset, label_dataset))

In [ ]:
# Repeat data and attach label
data_set  = origin_dataset.shuffle(global_value.SHUFFLE_SIZE).repeat(2)


# read data from path
data_set = data_set.map(format_function.process_image, num_parallel_calls=tf.data.AUTOTUNE)
# data_set = data_set.filter(lambda image, label: tf.math.not_equal(tf.size(image), 0))
data_set = data_set.map(format_function.augment_data, num_parallel_calls=tf.data.AUTOTUNE)
data_set = data_set.shuffle(global_value.SHUFFLE_SIZE)

# batch data
data_set = data_set.batch(global_value.BATCH_SIZE)

# # Set cache and prefetch to improve performance
data_set = data_set.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)

## Start train

# Train version 2


In [ ]:
# The embedding model
input_shape = [global_value.IMAGE_SIZE[0], global_value.IMAGE_SIZE[1], 3]
face_net_model = call_instance_FaceNet_with_last_isDense(input_shape, len(label_dict))
face_net_model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
    )
#----choose path to save per epoch
actual_epochs = 1
for i in range(100):
  last_save_path = "save_model/face_recognize_entropy{}".format(actual_epochs)
  if not os.path.exists(last_save_path):
    break
  actual_epochs += 1

# Load save
if (actual_epochs != 1):
  load_path = "save_model/face_recognize_entropy{}".format(actual_epochs-1)
  print(load_path)
  face_net_model = tf.keras.models.load_model(load_path)



# Normal train network
for i in range(global_value.EPOCHS):
  # Read data path from file
  path = os.path.join(os.getcwd(),"src","data","align_data_path.pkl")
  with open(path, 'rb') as f:
      path_image_align = pickle.load(f)
      path_image_align = file_function.get_data_path_with_limit(path_image_align,global_value.IMAGE_EACH_CLASS)
  path = os.path.join(os.getcwd(),"src","data","face+mask_data_path.pkl")
  with open(path, 'rb') as f:
      path_image_mask = pickle.load(f)
      path_image_mask = file_function.get_data_path_with_limit(path_image_mask,global_value.IMAGE_EACH_CLASS)
  # Combine data path

  path_image_align.extend(path_image_mask)
  random.shuffle(path_image_align)
  label_index =list()
  for path in path_image_align:
    label = path.split("/")[-2]
    label = label_dict[label]
    label_index.append(label)
  path_dataset = tf.data.Dataset.from_tensor_slices(path_image_align)
  label_dataset = tf.data.Dataset.from_tensor_slices(label_index)
  origin_dataset = tf.data.Dataset.zip((path_dataset, label_dataset))
  # Repeat data and attach label
  data_set  = origin_dataset.shuffle(global_value.SHUFFLE_SIZE).repeat(2)


  # read data from path
  data_set = data_set.map(format_function.process_image, num_parallel_calls=tf.data.AUTOTUNE)
  # data_set = data_set.filter(lambda image, label: tf.math.not_equal(tf.size(image), 0))
  data_set = data_set.map(format_function.augment_data, num_parallel_calls=tf.data.AUTOTUNE)
  data_set = data_set.shuffle(global_value.SHUFFLE_SIZE)

  # batch data
  data_set = data_set.batch(global_value.BATCH_SIZE)

  # Set cache and prefetch to improve performance
  data_set = data_set.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
  print("--------------------------big epoch {}--------------------------".format(actual_epochs))
  history = face_net_model.fit(
      data_set,
      epochs = 1
  )
  face_net_model.save("save_model/face_recognize_entropy{}".format(actual_epochs))
  with open("src/loss/face_recognize_entropy.txt", "a") as file_object:
    file_object.write("\n")
    file_object.write("epoch {}, loss {}, batch {}". format(actual_epochs, history.history['loss'], global_value.BATCH_SIZE))
  actual_epochs += 1

## test embedding

In [ ]:
# face_net_model = tf.keras.models.load_model("save_model/align_image_origin36")
# classify = Classify(face_net_model, format_function)
# database_embedding = classify.embedding_all_data_by_directory(os.path.join(os.getcwd(),"dataset","lfw"))
# classify.save_embedding_to_file(database_embedding, os.path.join(os.getcwd(),"encodings","encode_lfw_epoch36.pkl"))
# #Preprocess data
# test_dataset = tf.data.Dataset.list_files("dataset/lfw/*/*",shuffle=False)
# test_dataset = test_dataset.map(format_function.get_label_as_number, num_parallel_calls=tf.data.AUTOTUNE)
# test_dataset = test_dataset.map(format_function.process_image, num_parallel_calls=tf.data.AUTOTUNE)
# test_dataset = test_dataset.filter(lambda image, label: tf.math.not_equal(tf.size(image), 0))
# test_dataset = test_dataset.batch(20)

# # Accuracy
# print(classify.evaluate(test_dataset, database_embedding))

In [ ]:
# Load network
face_net_model = tf.keras.models.load_model("save_model/face_recognize_entropy47")
face_net_model =  convert_train_model_to_embedding(face_net_model)
#Preprocess data
test_dataset = tf.data.Dataset.list_files("dataset/10_person/*/*",shuffle=False)
test_dataset = test_dataset.map(format_function.get_label_as_number, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(format_function.process_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.filter(lambda image, label: tf.math.not_equal(tf.size(image), 0))
test_dataset = test_dataset.batch(20)
# Evaluate the network
results = face_net_model.predict(test_dataset)

# Save test embeddings for visualization in projector
np.savetxt("vecs.tsv", results, delimiter='\t')

out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for img, labels in tfds.as_numpy(test_dataset):
    [out_m.write(str(x) + "\n") for x in labels]
out_m.close()


try:
  from google.colab import files
  files.download('vecs.tsv')
  files.download('meta.tsv')
except:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>